#Simulación de flujo en tiempo real con DStreams

##Contexto:
Los DStreams de Spark Streaming permiten procesar datos en tiempo real, dividiéndolos en microbatches. Este ejercicio busca que comprendas cómo se procesan eventos de forma continua desde una fuente básica.

##Consigna:
Simula un flujo de datos conectándote a un socket TCP, procesa el texto recibido y calcula el conteo de palabras en tiempo real utilizando DStreams.

##Paso a paso:

Configurar un StreamingContext con un intervalo de 2 segundos.

Conectarse a un socket TCP usando socketTextStream().

Aplicar transformaciones:

1- flatMap() para separar palabras.

2- map() para crear pares (palabra, 1).

3- reduceByKey() para sumar ocurrencias.

4- Mostrar resultados usando pprint() o foreachRDD().collect().

5- Ejecutar el flujo con .start() y detenerlo con .awaitTermination() o un time.sleep() para simulación.

In [1]:
!apt-get install openjdk-11-jdk -y
!pip install pyspark


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-11-jre
  x11-utils
Suggested packages:
  libxt-doc openjdk-11-demo openjdk-11-source visualvm mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxt-dev libxtst6 libxxf86dga1 openjdk-11-jdk
  openjdk-11-jre x11-utils
0 upgraded, 10 newly installed, 0 to remove and 35 not upgraded.
Need to get 5,367 kB of archives.
After this operation, 15.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-core all 2.37-2build1 [1,041 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fonts-dejavu-extra all 2.37-2build1 [2,041 kB]
Get:3 http://archive.ubuntu.com/ubuntu jam

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"


In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import time

# Crear SparkContext y StreamingContext con batch de 2 segundos
sc = SparkContext("local[2]", "WordCount")
ssc = StreamingContext(sc, 2)

# Datos simulados: lista de líneas que se "emiten" en tiempo real
rdd_queue = []
lines = [
    "hola mundo",
    "hola spark",
    "spark streaming es genial",
    "hola mundo hola"
]

for line in lines:
    rdd_queue.append(sc.parallelize([line]))

# Crear DStream a partir de la cola de RDDs
input_stream = ssc.queueStream(rdd_queue)

# Transformaciones
words = input_stream.flatMap(lambda line: line.split())
pairs = words.map(lambda word: (word, 1))
word_counts = pairs.reduceByKey(lambda a, b: a + b)

# Mostrar resultados
word_counts.pprint()

# Iniciar streaming
ssc.start()

# Ejecutar por 10 segundos y luego detener
time.sleep(10)
ssc.stop(stopSparkContext=True, stopGraceFully=True)


/usr/local/lib/python3.12/dist-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


-------------------------------------------
Time: 2025-08-30 23:47:16
-------------------------------------------
('mundo', 1)
('hola', 1)

-------------------------------------------
Time: 2025-08-30 23:47:18
-------------------------------------------
('hola', 1)
('spark', 1)

-------------------------------------------
Time: 2025-08-30 23:47:20
-------------------------------------------
('streaming', 1)
('es', 1)
('genial', 1)
('spark', 1)

-------------------------------------------
Time: 2025-08-30 23:47:22
-------------------------------------------
('mundo', 1)
('hola', 2)

-------------------------------------------
Time: 2025-08-30 23:47:24
-------------------------------------------

-------------------------------------------
Time: 2025-08-30 23:47:26
-------------------------------------------

-------------------------------------------
Time: 2025-08-30 23:47:28
-------------------------------------------

